In [5]:
import requests

token = 'cd5b792d5b2d7a65e9c31fd710773253f2772f5d'

#search request to get all Mumbai stations
URL = "http://api.waqi.info/search/?keyword=Mumbai&token=cd5b792d5b2d7a65e9c31fd710773253f2772f5d"
r = requests.get(url = URL)
data = r.json()


In [6]:
#search request to get all Mumbai stations
URL = "http://api.waqi.info/search/?keyword=Mumbai&token=cd5b792d5b2d7a65e9c31fd710773253f2772f5d"
r = requests.get(url = URL)
data = r.json()


In [7]:
#create an array of stations
stations = [item['station'] for item in data['data']]
#create an array of station URLs to be incorporated into each station-specific request
urls = [ item['url'] for item in stations ]
urls

['india/mumbai/deonar',
 'india/mumbai/bandra-kurla-complex',
 'india/mumbai/powai',
 'india/mumbai/kurla',
 'india/mumbai/chhatrapati-shivaji-intl.-airport-t2',
 'india/mumbai/sion',
 'india/mumbai/chakala-andheri-east',
 'india/mumbai/vile-parle-west',
 'india/mumbai/worli',
 'india/mumbai/us-consulate',
 'india/mumbai/siddharth-nagar-worli',
 'india/mumbai/bandra',
 'india/mumbai/colaba',
 'india/mumbai/vasai-west',
 'india/navi-mumbai/nerul',
 'india/navi-mumbai/mahape',
 'india/mumbai/nmmc-airoli',
 'india/mumbai/borivali-east',
 'india/navi-mumbai/sector-19a-nerul']

In [8]:
coords = [stations[i]['geo'] for i in range(len(stations))]
import numpy as np

npc = np.array(coords)
print(npc.max(axis=0))
print(npc.min(axis=0))
#print(npc)

[19.3832 73.0325]
[18.91      72.8128113]


In [9]:
#get current data based on each URL
stations_data = []
for item in urls:
  url = "https://api.waqi.info/feed/{url}/?token={token}".format(token=token, url=item)
  r = requests.get(url = url)
  data = r.json()
  stations_data.append(data)
  

In [10]:
#example for one station data
stations_data[2]['data']

{'aqi': 147,
 'idx': 12459,
 'attributions': [{'url': 'http://cpcb.nic.in/',
   'name': 'CPCB - India Central Pollution Control Board',
   'logo': 'India-CPCB.png'},
  {'url': 'https://waqi.info/', 'name': 'World Air Quality Index Project'}],
 'city': {'geo': [19.1375, 72.915056],
  'name': 'Powai, Mumbai, India',
  'url': 'https://aqicn.org/city/india/mumbai/powai',
  'location': ''},
 'dominentpol': 'pm25',
 'iaqi': {'co': {'v': 6.2},
  'dew': {'v': 25},
  'h': {'v': 90.74666666666667},
  'no2': {'v': 13.8},
  'o3': {'v': 16.4},
  'p': {'v': 1011},
  'pm10': {'v': 89},
  'pm25': {'v': 147},
  'so2': {'v': 5},
  't': {'v': 29},
  'w': {'v': 0.49666666666666665},
  'wg': {'v': 9.7}},
 'time': {'s': '2023-04-11 02:00:00',
  'tz': '+05:30',
  'v': 1681178400,
  'iso': '2023-04-11T02:00:00+05:30'},
 'forecast': {'daily': {'o3': [{'avg': 15,
     'day': '2023-04-09',
     'max': 59,
     'min': 1},
    {'avg': 15, 'day': '2023-04-10', 'max': 61, 'min': 2},
    {'avg': 14, 'day': '2023-04-1

In [11]:
import pandas as pd

df = pd.DataFrame()

data_ar = []

indicators = ['co', 'no2', 'so2', 'o3', 'pm10', 'pm25']

for station_data in stations_data:
    aq = station_data['data']['iaqi']
    aqi = station_data['data']['aqi']
    row = {}
    row.update({'station':station_data['data']['city']['name']})
    row.update({'time':station_data['data']['time']['s']})
    #assign non existent values with None
    for ind in indicators:
        if (ind in aq.keys()) and (aq[ind] != None):
            row.update({ind:aq[ind]['v']})
    row.update({'aqi':aqi})
    data_ar.append(row)

In [12]:
pd.DataFrame(data_ar)

,station,time,co,no2,so2,o3,pm10,pm25,aqi
0,"Deonar, Mumbai, India",2023-04-07 16:00:00,4.1,8.5,17.9,30.3,116.0,181.0,181
1,"Bandra Kurla Complex, Mumbai, India",2023-04-09 19:00:00,9.5,44.0,0.5,12.9,112.0,169.0,169
2,"Powai, Mumbai, India",2023-04-11 02:00:00,6.2,13.8,5.0,16.4,89.0,147.0,147
3,"Kurla, Mumbai, India",2023-04-11 01:00:00,5.5,12.8,13.0,16.0,142.0,81.0,142
4,"Chhatrapati Shivaji Intl. Airport (T2), Mumbai...",2023-04-11 01:00:00,17.6,18.0,4.4,9.5,119.0,131.0,131
5,"Sion, Mumbai, India",2023-04-11 01:00:00,6.8,1.7,3.1,1.9,83.0,129.0,129
6,"Chakala-Andheri East, Mumbai, India",2023-04-11 01:00:00,18.1,6.8,4.3,10.7,60.0,129.0,129
7,"Vile Parle West, Mumbai, India",2023-04-11 01:00:00,9.6,3.6,25.0,1.5,70.0,102.0,102
8,"Worli, Mumbai, India",2023-04-11 01:00:00,9.6,4.2,1.2,0.2,52.0,99.0,99
9,"Mumbai US Consulate, India (मुंबई अमेरिकी वाणि...",2023-04-11 01:00:00,NaN,NaN,NaN,NaN,NaN,89.0,89
